<a href="https://colab.research.google.com/github/thingumajig/colab-experiments/blob/master/russian_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install spacy-udpipe
!pip install pymorphy2
!pip install nltk
!pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 8.0MB 2.3MB/s 


In [4]:
import pymorphy2
import spacy_udpipe

from spacy.symbols import *

from nltk import Tree
def tok_format(t):
    return f'{t.orth_}-{t.dep_}'

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(tok_format(node), [to_nltk_tree(child) for child in node.children])
    else:
        return tok_format(node)

def print_tree(node):
  to_nltk_tree(node).pretty_print()

spacy_udpipe.download("ru") # download English model


nlp = spacy_udpipe.load("ru")
# nlp.add_pipe(nlp.create_pipe('sentencizer'))




Successfully downloaded the pretrained UDPipe model for the 'ru' language


In [39]:
morph = pymorphy2.MorphAnalyzer()
all_t = morph.parse('условия')
print(all_t)
p = all_t[0]
print(p.tag)

v = {p.tag.case}
print(v)
a = morph.parse('городской')[0]
a = a.inflect(v)

print(a)



[Parse(word='условия', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='условие', score=0.75, methods_stack=((<DictionaryAnalyzer>, 'условия', 76, 2),)), Parse(word='условия', tag=OpencorporaTag('NOUN,inan,neut plur,nomn'), normal_form='условие', score=0.125, methods_stack=((<DictionaryAnalyzer>, 'условия', 76, 13),)), Parse(word='условия', tag=OpencorporaTag('NOUN,inan,neut plur,accs'), normal_form='условие', score=0.125, methods_stack=((<DictionaryAnalyzer>, 'условия', 76, 18),))]
NOUN,inan,neut sing,gent
{'gent'}
Parse(word='городского', tag=OpencorporaTag('ADJF masc,sing,gent'), normal_form='городской', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'городского', 575, 1),))


In [5]:
%%time
text = '''
избрание и досрочное прекращение полномочий Единоличного исполнительного органа, Коллегиального исполнительного органа (Правления), утверждение условий договора с Единоличным исполнительным органом, членами Правления, принятие решения (в том числе об определении и порядке выплаты) о вознаграждении, поощрении, премировании, компенсациях и иных выплатах, в том числе стимулирующего и компенсационного характера Единоличному исполнительному органу и членам Правления (за исключением ключевых показателей эффективности (KPI), утверждение условий расторжения договора с Единоличным исполнительным органом, членами Правления, а также принятие решения о передаче полномочий Единоличного исполнительного органа коммерческой организации или индивидуальному предпринимателю (управляющему), утверждения такого управляющего и условий договора с ним;
'''
text = '''
Сухопутные войска Великобритании в декабре 2019 года испытали одну из модификаций танка Challenger 2 (Streetfighter II) центре боевой подготовки в городских условиях Коупхилл-Даун на военном полигоне в Солсбери (Англия), сообщает Jane's Defence Weekly. Соответствующий ролик выложен на YouTube.
Британский журнал отмечает, что городской танк Streetfighter II получил распределенную по его корпусу систему инфракрасных и электрооптических датчиков IronVision израильской компании Elbit Systems, позволяющую отображать оперативную обстановку вокруг боевой машины на нашлемный дисплей танкиста.
'''
#TODO
s1 = 'одобрение сделок с недвижимостью'
s2 = 'утверждение сделок с имуществом'
s3 = 'одобрение сделок за исключением сделок с недвижимостью'

doc = nlp(text, disable=['ner'])


CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 106 ms


In [37]:
print('Sentences:'+'='*10)
for sent in doc.sents:
    s = str(sent).strip()
    print(s)

print('Tokens:'+'='*10)
for token in doc:
    # print(f'{token.text}\t{token.lemma_}\t{token.pos_}\t{token.dep_}:{token.dep}')
    s = ''
    cc = 0
    for t in token.subtree:
      s += str(t)+' '
      cc +=1
    # if cc>1:
    print(f'[{token.dep_}-{token.dep}-{token.pos_}--{token.text}]\t{s}')
    if token.n_lefts + token.n_rights > 0:
      print_tree(token)

np_labels_full = {nsubj, nsubjpass, dobj, iobj, pobj, csubj, csubjpass, attr,
                  obj, nmod}  # obl, nmod,  Probably others too


def iter_nps(doc):
    for word in doc:
      if word.dep in np_labels_full:
        yield word

exclude_labels = {nmod, acl}           
def iter_nps_str(doc):
  s = ''
  for np in iter_nps(doc):
    
    excluded = set()
    for child in np.children:
      if child is not np and child.dep in exclude_labels:
        excluded.add(child)
        for t in child.subtree:
          excluded.add(t)
      elif child.dep_ == 8110129090154140942: # child.dep == case
        excluded.add(child)
    
    for t in np.subtree:
      # if t.head.dep == nmod
      if t not in excluded:
        s += str(t)+' '
    yield s.strip()
    s = ''

print('='*20)

for np in iter_nps_str(doc):
  print(np)

Sentences:==========
Сухопутные войска Великобритании в декабре 2019 года испытали одну из модификаций танка Challenger 2 (Streetfighter II) центре боевой подготовки в городских условиях Коупхилл-Даун на военном полигоне в Солсбери (Англия), сообщает Jane's Defence Weekly.
Соответствующий ролик выложен на YouTube.
Британский журнал отмечает, что городской танк Streetfighter II получил распределенную по его корпусу систему инфракрасных и электрооптических датчиков IronVision израильской компании Elbit Systems, позволяющую отображать оперативную обстановку вокруг боевой машины на нашлемный дисплей танкиста.
Tokens:==========
[amod-402-ADJ--Сухопутные]	Сухопутные 
[nsubj-429-NOUN--войска]	Сухопутные войска Великобритании 
                войска-nsubj                
        _____________|______________         
Сухопутные-amod              Великобритании-
                                   nmod     

[nmod-426-PROPN--Великобритании]	Великобритании 
[case-8110129090154140942-ADP--в]	в 
[ob